In [2]:
#卷积神经网络我们一定要熟悉每一层过后的维度
# 98 = 100 - 3 + 1
# 有 10*5*3*3 个要优化的权重
# 有10个卷积核，每个卷积核的channel与input_channel相同，是5，卷积核的长宽是kernel_size
import torch
in_channels, out_channels= 5, 10 
width, height = 100, 100 
kernel_size = 3
batch_size = 1
input = torch.randn(batch_size, in_channels,
                    width,
                    height)
conv_layer = torch.nn.Conv2d(in_channels, out_channels,
                             kernel_size=kernel_size)
output = conv_layer(input)
print(input.shape) 
print(output.shape) 
print(conv_layer.weight.shape)

torch.Size([1, 5, 100, 100])
torch.Size([1, 10, 98, 98])
torch.Size([10, 5, 3, 3])


In [4]:
# padding
import torch
input = [3,4,6,5,7, 2,4,6,8,2, 1,6,7,8,4, 9,7,4,6,2, 3,7,5,4,1]
input = torch.Tensor(input).view(1, 1, 5, 5)
conv_layer = torch.nn.Conv2d(1, 1, kernel_size=3, padding=1, bias=False)
kernel = torch.Tensor([1,2,3,4,5,6,7,8,9]).view(1, 1, 3, 3)
conv_layer.weight.data = kernel.data
output = conv_layer(input) 
print(output)

tensor([[[[ 91., 168., 224., 215., 127.],
          [114., 211., 295., 262., 149.],
          [192., 259., 282., 214., 122.],
          [194., 251., 253., 169.,  86.],
          [ 96., 112., 110.,  68.,  31.]]]], grad_fn=<ThnnConv2DBackward>)


In [5]:
# stride
import torch
input = [3,4,6,5,7, 2,4,6,8,2, 1,6,7,8,4, 9,7,4,6,2, 3,7,5,4,1]
input = torch.Tensor(input).view(1, 1, 5, 5)
conv_layer = torch.nn.Conv2d(1, 1, kernel_size=3, stride=2, bias=False)
kernel = torch.Tensor([1,2,3,4,5,6,7,8,9]).view(1, 1, 3, 3)
conv_layer.weight.data = kernel.data
output = conv_layer(input)
print(output)

tensor([[[[211., 262.],
          [251., 169.]]]], grad_fn=<ThnnConv2DBackward>)


In [6]:
# 最大池化
import torch
input = [3,4,6,5, 2,4,6,8, 1,6,7,8, 9,7,4,6,
        ]
input = torch.Tensor(input).view(1, 1, 4, 4)
maxpooling_layer = torch.nn.MaxPool2d(kernel_size=2)
output = maxpooling_layer(input)
print(output)

tensor([[[[4., 8.],
          [9., 8.]]]])


In [12]:
# 平均池化
import torch
input = [3,4,6,5, 2,4,6,8, 1,6,7,8, 9,7,4,6,
        ]
input = torch.Tensor(input).view(1, 1, 4, 4)
maxpooling_layer = torch.nn.AvgPool2d(kernel_size = 2)
output = maxpooling_layer(input)
print(output)

tensor([[[[3.2500, 6.2500],
          [5.7500, 6.2500]]]])


In [9]:
# 对于pytorch view函数
a=torch.Tensor([[[1,2,3],[4,5,6]]])
b=torch.Tensor([1,2,3,4,5,6])

print(a.view(1,6))
print(b.view(1,6))
print(a.view(2,-1))   # -1会自动帮忙算

tensor([[1., 2., 3., 4., 5., 6.]])
tensor([[1., 2., 3., 4., 5., 6.]])
tensor([[1., 2., 3.],
        [4., 5., 6.]])


In [11]:
# 卷积神经网络
import torch
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
 
# prepare dataset
 
batch_size = 64
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
 
train_dataset = datasets.MNIST(root='../dataset/mnist/', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
test_dataset = datasets.MNIST(root='../dataset/mnist/', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)
 
# design model using class
 
 
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = torch.nn.Conv2d(10, 20, kernel_size=5)
        self.pooling = torch.nn.MaxPool2d(2)
        self.fc = torch.nn.Linear(320, 10)
 
 
    def forward(self, x):
        # flatten data from (n,1,28,28) to (n, 784)
        # mnist是黑白图，只有一个channel
        batch_size = x.size(0)
        x = F.relu(self.pooling(self.conv1(x)))
        x = F.relu(self.pooling(self.conv2(x)))
        x = x.view(batch_size, -1) # -1 此处自动算出的是320
        # print(x.shape) 此处自动酸楚是320

        
        #print(batch_size)是64
        x = self.fc(x)
        # 后面用的CrossENtropyLoss，即使是分类也不需要经过softmax
        return x
 
 
model = Net()
 
# construct loss and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
 
# training cycle forward, backward, update
 
 
def train(epoch):
    running_loss = 0.0
    for batch_idx, data in enumerate(train_loader, 0):
        inputs, target = data
        optimizer.zero_grad()
 
        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
 
        running_loss += loss.item()
        if batch_idx % 300 == 299:
            print('[%d, %5d] loss: %.3f' % (epoch+1, batch_idx+1, running_loss/300))
            running_loss = 0.0
 
 
def test():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('accuracy on test set: %d %% ' % (100*correct/total))
 
 
if __name__ == '__main__':
    for epoch in range(10):
        train(epoch)
        test()

torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size

KeyboardInterrupt: 

In [22]:
# 采用GoogleNet：利用inception，实现训练中通过权重的改变“选择”合适的路

# 我们发现先比如前面那个直接CNN，这个loss下降的快，但是训练的时间长
import torch
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
 
# prepare dataset
 
batch_size = 64
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
 
train_dataset = datasets.MNIST(root='../dataset/mnist/', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
test_dataset = datasets.MNIST(root='../dataset/mnist/', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)


class InceptionA(torch.nn.Module):
    def __init__(self, in_channels):
        super(InceptionA, self).__init__()
        self.branch1x1 = torch.nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch5x5_1 = torch.nn.Conv2d(in_channels,16, kernel_size=1)
        self.branch5x5_2 = torch.nn.Conv2d(16, 24, kernel_size=5, padding=2)
        self.branch3x3_1 = torch.nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch3x3_2 = torch.nn.Conv2d(16, 24, kernel_size=3, padding=1)
        self.branch3x3_3 = torch.nn.Conv2d(24, 24, kernel_size=3, padding=1)
        self.branch_pool = torch.nn.Conv2d(in_channels, 24, kernel_size=1) 
    def forward(self, x):
        branch1x1 = self.branch1x1(x)
        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)
        branch3x3 = self.branch3x3_1(x) 
        branch3x3 = self.branch3x3_2(branch3x3) 
        branch3x3 = self.branch3x3_3(branch3x3)
        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)
        outputs = [branch1x1, branch5x5, branch3x3, branch_pool]
        return torch.cat(outputs, dim=1)
class Net(torch.nn.Module): 
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 10, kernel_size=5) 
        self.conv2 = torch.nn.Conv2d(88, 20, kernel_size=5)
        self.incep1 = InceptionA(in_channels=10) 
        self.incep2 = InceptionA(in_channels=20)
        self.mp = torch.nn.MaxPool2d(2) 
        self.fc = torch.nn.Linear(1408, 10)
    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x))) 
        x = self.incep1(x)
        x = F.relu(self.mp(self.conv2(x))) 
        x = self.incep2(x)
        x = x.view(in_size, -1)
        x = self.fc(x)
        return x


Model = Net()
 
# construct loss and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(Model.parameters(), lr=0.01, momentum=0.5)
 
# training cycle forward, backward, update
 
 
def train(epoch):
    running_loss = 0.0
    for batch_idx, data in enumerate(train_loader, 0):
        inputs, target = data
        optimizer.zero_grad()
 
        outputs = Model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
 
        running_loss += loss.item()
        if batch_idx % 300 == 299:
            print('[%d, %5d] loss: %.3f' % (epoch+1, batch_idx+1, running_loss/300))
            running_loss = 0.0
 
 
def test():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = Model(images)
            _, predicted = torch.max(outputs.data, dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('accuracy on test set: %d %% ' % (100*correct/total))
 
 
if __name__ == '__main__':
    for epoch in range(10):
        train(epoch)
        test()

[1,   300] loss: 0.923
[1,   600] loss: 0.195
[1,   900] loss: 0.146
accuracy on test set: 96 % 
[2,   300] loss: 0.112
[2,   600] loss: 0.104
[2,   900] loss: 0.089
accuracy on test set: 97 % 
[3,   300] loss: 0.085
[3,   600] loss: 0.077
[3,   900] loss: 0.073
accuracy on test set: 97 % 
[4,   300] loss: 0.070
[4,   600] loss: 0.065
[4,   900] loss: 0.063
accuracy on test set: 98 % 
[5,   300] loss: 0.058
[5,   600] loss: 0.059
[5,   900] loss: 0.056
accuracy on test set: 98 % 
[6,   300] loss: 0.050
[6,   600] loss: 0.051
[6,   900] loss: 0.054
accuracy on test set: 98 % 
[7,   300] loss: 0.045
[7,   600] loss: 0.045
[7,   900] loss: 0.051
accuracy on test set: 98 % 
[8,   300] loss: 0.045
[8,   600] loss: 0.044


KeyboardInterrupt: 

In [24]:
# 基础RNN
import torch
batch_size = 1
seq_len = 3
input_size = 4
hidden_size = 2
cell = torch.nn.RNNCell(input_size=input_size, hidden_size=hidden_size)
# (seq, batch, features)
dataset = torch.randn(seq_len, batch_size, input_size) 
hidden = torch.zeros(batch_size, hidden_size)
for idx, input in enumerate(dataset): 
    print('=' * 20, idx, '=' * 20) 
    print('Input size: ', input.shape)
    hidden = cell(input, hidden)
    print('outputs size: ', hidden.shape) 
    print(hidden)

==================== 0 ====================
Input size:  torch.Size([1, 4])
outputs size:  torch.Size([1, 2])
tensor([[-0.8296,  0.4656]], grad_fn=<TanhBackward>)
==================== 1 ====================
Input size:  torch.Size([1, 4])
outputs size:  torch.Size([1, 2])
tensor([[-0.6691, -0.7493]], grad_fn=<TanhBackward>)
==================== 2 ====================
Input size:  torch.Size([1, 4])
outputs size:  torch.Size([1, 2])
tensor([[-0.9605, -0.1718]], grad_fn=<TanhBackward>)


In [29]:
 # RNN我们理解他的关键就是懂每一层后的维度。
import torch
batch_size = 1 
seq_len = 3 
input_size = 4 
hidden_size = 2 
num_layers = 1
cell = torch.nn.RNN(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers)
#定义后里面的权重矩阵包括偏置都会被随机初始化的，然后后面通过loss和optimin进行优化
# (seqLen, batchSize, inputSize)
print(cell)
inputs = torch.randn(seq_len, batch_size, input_size) 
hidden = torch.zeros(num_layers, batch_size, hidden_size)
out, hidden = cell(inputs, hidden)
print('Output size:', out.shape) 
print('Output:', out)
print('Hidden size: ', hidden.shape) 
print('Hidden: ', hidden)

RNN(4, 2)
Output size: torch.Size([3, 1, 2])
Output: tensor([[[-0.9323,  0.2430]],

        [[-0.7208, -0.0095]],

        [[-0.7323, -0.6566]]], grad_fn=<StackBackward>)
Hidden size:  torch.Size([1, 1, 2])
Hidden:  tensor([[[-0.7323, -0.6566]]], grad_fn=<StackBackward>)


In [71]:
# demo train "hello" to "ohlol"
# 上面那个报错，这个就正确
import torch

input_size = 4
hidden_size = 4
num_layers = 1
batch_size = 1
seq_len = 5

idx2char = ['e', 'h', 'l', 'o']
x_data = [1, 0, 2, 2, 3]
y_data = [3, 1, 2, 3, 2]

one_hot_lookup = [[1, 0, 0, 0],
                  [0, 1, 0, 0],
                  [0, 0, 1, 0],
                  [0, 0, 0, 1]]

x_one_hot = [one_hot_lookup[x] for x in x_data]

inputs = torch.Tensor(x_one_hot).view(-1, batch_size, input_size)
labels = torch.LongTensor(y_data).view(-1, 1)

class Model(torch.nn.Module):
    def __init__(self, input_size, hidden_size, batch_size, num_layers=1):
        super().__init__()
        self.batch_size = batch_size
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.rnncell = torch.nn.RNNCell(input_size=self.input_size,
                                        hidden_size=self.hidden_size)

    def forward(self, input, hidden):
        hidden = self.rnncell(input, hidden)
        return hidden

    def init_hidden(self):
        return torch.zeros(self.batch_size, self.hidden_size)

net = Model(input_size, hidden_size, batch_size)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.1)

for epoch in range(30):
    loss = 0
    optimizer.zero_grad()
    hidden = net.init_hidden()
    print('Predicted string:', end='')
    for input, label in zip(inputs, labels):
        hidden = net.forward(input, hidden)
        #print(hidden)
        loss += criterion(hidden, label)
        _, idx = hidden.max(dim=1)
        print(idx2char[idx.item()], end='')
    loss.backward()
    optimizer.step()
    print(', Epoch [%d/30] loss=%.4f' % (epoch+1, loss.item()))


Predicted string:lllle, Epoch [1/30] loss=7.3901
Predicted string:lllle, Epoch [2/30] loss=6.2142
Predicted string:ollll, Epoch [3/30] loss=5.2851
Predicted string:ollll, Epoch [4/30] loss=4.6927
Predicted string:ollll, Epoch [5/30] loss=4.3786
Predicted string:ollll, Epoch [6/30] loss=4.1536
Predicted string:ollll, Epoch [7/30] loss=3.9559
Predicted string:ohlll, Epoch [8/30] loss=3.7364
Predicted string:ohlol, Epoch [9/30] loss=3.4777
Predicted string:ohlol, Epoch [10/30] loss=3.2137
Predicted string:ohlol, Epoch [11/30] loss=3.0235
Predicted string:ohlol, Epoch [12/30] loss=2.8969
Predicted string:ohlol, Epoch [13/30] loss=2.7369
Predicted string:ohlol, Epoch [14/30] loss=2.5757
Predicted string:ohlol, Epoch [15/30] loss=2.4354
Predicted string:ohlol, Epoch [16/30] loss=2.3133
Predicted string:ohlol, Epoch [17/30] loss=2.2056
Predicted string:ohlol, Epoch [18/30] loss=2.1143
Predicted string:ohlol, Epoch [19/30] loss=2.0425
Predicted string:ohlol, Epoch [20/30] loss=1.9891
Predicted